# Librerías

In [ ]:
#!pip install pandas scikit-learn numpy matplotlib
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from google.colab import files

# Importar los datos

In [ ]:
uploaded = files.upload()

In [ ]:

# Supongamos que tu archivo se llama dataset.xlsx
df = pd.read_excel("dataset.xlsx", index_col=0, parse_dates=True)

# Información del dataframe
df.info()
print(df.head())
# Ver distribución de la columna 'diagnostico'
df['ciclo'].value_counts()

# Crear set de entrenamiento

In [ ]:
# X: entradas, Y: salida a predecir
X = df.drop('ciclo', axis=1) # Drop the 'ciclo' column from df
Y = df['ciclo'] # Use the correct column name for Y from df

# Partición
train_size = int(0.5 * len(X))

# Entrenamiento: primeros 70%
x_train = X.iloc[:train_size]
y_train = Y.iloc[:train_size]

# Test: últimos 30%
x_test = X.iloc[train_size:]
y_test = Y.iloc[train_size:]

# Imprimir tamaños resultantes
print('Tamaño set de entrenamiento: ', x_train.shape, y_train.shape)
print('Tamaño set de prueba: ', x_test.shape, y_test.shape)

# Entrenar bosque aleatorio simple
obb_score se realiza sobre la muestra out of the bag

In [ ]:
bosque = RandomForestClassifier(random_state=123,
                                oob_score=True)
bosque.fit(x_train,y_train)

bosque.oob_score_
len(bosque.estimators_)
bosque.estimators_[50].get_n_leaves()


print("OOB Score:", bosque.oob_score_)
print("Número de árboles:", len(bosque.estimators_))
print("Número de hojas en el árbol 50:", bosque.estimators_[50].get_n_leaves())

In [ ]:
#Matriz de clasificación de probabilidades
bosque.oob_decision_function_.shape
bosque.oob_decision_function_
print("Shape de oob_decision_function_:", bosque.oob_decision_function_.shape)
print("Valores de oob_decision_function_:\n", bosque.oob_decision_function_)

METRICAS
Precisión (precision): mide qué proporción de las predicciones positivas fueron correctas. Es decir, de todas las veces que el modelo predijo “Y1”, cuántas realmente fueron “Y1”.

Recall (recall): indica qué proporción de los positivos reales fueron detectados por el modelo. Es decir, de todos los casos que realmente fueron “Y1”, cuántos predijo correctamente.

Puntaje F1 (puntaje_f1): es la media armónica entre precisión y recall. Resume en un solo número el balance entre ambos, útil cuando las clases están desbalanceadas.

In [ ]:
#Metricas de evaluación a partir de la matriz de confusión

# Probabilidades OOB
oob_probs = bosque.oob_decision_function_

# Convertir probabilidades a categorías predichas
oob_preds = np.argmax(oob_probs, axis=1) # Seleccionar probabilidad más alta

# Calcular precision, recall y puntaje f1
precision = precision_score(y_train, oob_preds)
recall = recall_score(y_train, oob_preds)
puntaje_f1 = f1_score(y_train, oob_preds)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Puntaje F1: {puntaje_f1:.4f}')

# Entrenamiento del modelo variando parametros

In [ ]:
# Entrenar bosque con diferentes valores de "n_estimators" número de árboles

N_ESTIMADORES = np.arange(10,301,10)
f1_scores = []

for n_arboles in N_ESTIMADORES:
    print(f'Entrenando con {n_arboles} árboles...')
    bosque = RandomForestClassifier(n_estimators=n_arboles,
                                    random_state=123,
                                    oob_score=True)
    bosque.fit(x_train,y_train)
    oob_probs = bosque.oob_decision_function_
    oob_preds = np.argmax(oob_probs, axis=1)
    f1_scores.append(f1_score(y_train, oob_preds))

# Graficar f1_scores vs. n_arboles
plt.plot(N_ESTIMADORES, f1_scores)
plt.xlabel('Número de árboles')
plt.ylabel('F1-score');

In [ ]:
# Con N arboles definidos y probar con max_depth  profundidad máxima
MAX_DEPTHS = np.arange(2,61,2)
f1_scores = []

for max_depth in MAX_DEPTHS:
    print(f'Entrenando con max_depth = {max_depth} ...')
    bosque = RandomForestClassifier(n_estimators=300,
                                    max_depth=max_depth,
                                    random_state=123,
                                    oob_score=True)
    bosque.fit(x_train,y_train)
    oob_probs = bosque.oob_decision_function_
    oob_preds = np.argmax(oob_probs, axis=1)
    f1_scores.append(f1_score(y_train, oob_preds))

# Graficar f1_scores vs. max_depth
plt.plot(MAX_DEPTHS, f1_scores)
plt.xlabel('Máxima profundidad')
plt.ylabel('F1-score');

In [ ]:
# Con N arboles definidos y probar con max_features número máximo de características
MAX_FEATURES = np.arange(1,len(x_train.columns))
f1_scores = []

for max_features in MAX_FEATURES:
    print(f'Entrenando con max_features = {max_features} ...')
    bosque = RandomForestClassifier(n_estimators=25,
                                    max_features = max_features,
                                    random_state=123,
                                    oob_score=True)
    bosque.fit(x_train,y_train)
    oob_probs = bosque.oob_decision_function_
    oob_preds = np.argmax(oob_probs, axis=1)
    f1_scores.append(f1_score(y_train, oob_preds))

# Graficar f1_scores vs. max_features
plt.plot(MAX_FEATURES, f1_scores)
plt.xlabel('Máximo número de características')
plt.ylabel('F1-score');

# Predicciones

In [ ]:
# Entrenemos el bosque con los parámetros
bosque = RandomForestClassifier(n_estimators=300,   #cambiar
                                max_depth=8,       #cambiar
                                max_features=6,    #cambiar
                                random_state=123,
                                oob_score=True)
bosque.fit(x_train,y_train)

In [ ]:
# Finalmente predicciones
preds = bosque.predict(x_test)
preds

cm = confusion_matrix(y_test, preds, labels=[0, 1])


sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

print("Recall:", recall_score(y_test, preds))
print("Precision:", precision_score(y_test, preds))
print("F1 Score:", f1_score(y_test, preds))

In [ ]:
# Suponiendo que y_test y preds están en el mismo orden temporal
plt.figure(figsize=(12,5))

# Graficar valores reales
plt.plot(y_test.reset_index(drop=True), label='Real', marker='o')

# Graficar predicciones
plt.plot(preds, label='Predicho', marker='x')

plt.xlabel('Periodo')
plt.ylabel('Valor')
plt.title('Comparación de valores reales vs predichos')
plt.legend()
plt.show()

In [ ]:
#RandomForestClassifier entrenado
importancia = pd.DataFrame({
    'Caracteristica': x_train.columns,
    'Importancia': bosque.feature_importances_
})

# Ordenar de mayor a menor
importancia = importancia.sort_values(by='Importancia', ascending=False)
print(importancia)